In [53]:
import numpy as np
import pandas as pd
import tensorflow as tf



In [54]:
user_rating = pd.read_csv('../../dataset/user_rating.csv')
place_num = len(user_rating.Place_Id.unique())
n_users = len(user_rating.User_Id.unique())



In [55]:
df_destination = pd.read_csv('../../dataset/new_bali_dataset.csv', delimiter=';', header=0)
df_destination['price'].fillna(0, inplace=True)
df_destination['category'].fillna('unknown', inplace=True)

In [56]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(user_rating, test_size=0.2, random_state=42)

In [57]:
def create_model():
    # creating destination embedding path
    destination_input = Input(shape=[1])
    destination_embedding = Embedding(place_num+1, 5)(destination_input)
    destination_flat = Flatten()(destination_embedding)

    # creating user embedding path
    user_input = Input(shape=[1])
    user_embedding = Embedding(n_users+1, 5)(user_input)
    user_flat = Flatten()(user_embedding)

    # concatenate features
    conc = Concatenate()([destination_flat, user_flat])

    # add fully-connected-layers
    x = Dense(128, activation='relu')(conc)
    x = Dense(64, activation='relu')(x)
    output = Dense(1)(x)

    # Create model and compile it
    model = Model([user_input, destination_input], output)
    model.compile('adam', 'mean_squared_error')
    history = model.fit([train.User_Id, train.Place_Id], train.Place_Ratings, epochs=20)
    return history, model

Predict

In [58]:
# bali_dataset = pd.read_csv('../../dataset/new_bali_dataset.csv', delimiter=';')
# recommended_places = bali_dataset.loc[recommended_tourism_ids, 'place']
# recommended_places

In [59]:
history, model = create_model()


Epoch 1/20
229/229 [==============================] - 3s 3ms/step - loss: 3.1715
Epoch 2/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8766
Epoch 3/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8366
Epoch 4/20
229/229 [==============================] - 1s 3ms/step - loss: 1.8203
Epoch 5/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8094
Epoch 6/20
229/229 [==============================] - 1s 3ms/step - loss: 1.8118
Epoch 7/20
229/229 [==============================] - 1s 3ms/step - loss: 1.8002
Epoch 8/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8081
Epoch 9/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8075
Epoch 10/20
229/229 [==============================] - 1s 3ms/step - loss: 1.7943
Epoch 11/20
229/229 [==============================] - 1s 4ms/step - loss: 1.7866
Epoch 12/20
229/229 [==============================] - 1s 4ms/step - loss: 1.7820
Epoch 13/20
229/229 [====

In [73]:
def create_final_data(list_of_idx):
    new_df = df_destination[df_destination.index.isin(list_of_idx)].copy()
    list_of_dest = []

    for row in new_df.iterrows():
        dest_dict = {}
        dest_dict['place'] = row[1]['place']
        dest_dict['url'] = row[1]['url']
        dest_dict['address'] = row[1]['address']
        dest_dict['is_accessibility'] = row[1]['is_accessibility']
        dest_dict['rating'] = row[1]['rating']
        dest_dict['n_reviews'] = row[1]['n_reviews']
        dest_dict['price'] = row[1]['price']
        dest_dict['category'] = row[1]['category']
        dest_dict['description'] = row[1]['description']
        list_of_dest.append(dest_dict)
    
    return list_of_dest

In [75]:
def might_like(idx_user, model=model):
    tourism_data = np.array(list(set(user_rating.Place_Id)))
    user = np.array([idx_user]*len(tourism_data))

    
    predictions = model.predict([user, tourism_data])
    predictions = np.array([a[0] for a in predictions])
    
    recommended_tourism_idx = (-predictions).argsort()[:10]
    
    #convert index to data place
    data = create_final_data(recommended_tourism_idx)
    return data

In [76]:
idx_user = 5
get_recom = might_like(idx_user)
get_recom

7/7 [==============================] - 0s 1ms/step


[{'place': 'Kedungu beach Bali', 'rating': 4.4},
 {'place': 'Pantai Ciwa', 'rating': 4.5},
 {'place': 'PANTAI SERIRIT', 'rating': 4.6},
 {'place': 'Bali Budaya Cultural Village', 'rating': 4.6},
 {'place': 'Taman Werdhi Budaya Art Centre', 'rating': 4.6},
 {'place': 'Pantai Pandawa', 'rating': 4.6},
 {'place': 'Patung Catur Muka', 'rating': 4.7},
 {'place': 'Pantai Mengening', 'rating': 4.7},
 {'place': 'Brahmavihara Arama', 'rating': 4.7},
 {'place': 'Uluwatu Temple', 'rating': 4.6}]